In [1]:
from elasticsearch import Elasticsearch
from datetime import datetime as dt
import pandas as pd
import numpy as np
from csv import DictReader

### Creamos la conexion con elastic

In [2]:
ELASTIC_PASSWORD = "k3cwTESnoemDc1YCxO-s" #ane


client = Elasticsearch(
    "https://localhost:9200",
    ca_certs="/home/bdata2/elasticsearch-8.4.3/config/certs/http_ca.crt",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

client.info()

ObjectApiResponse({'name': 'bdata2-VirtualBox', 'cluster_name': 'elasticsearch', 'cluster_uuid': '6XmqEv5uSK-hr779sbmGdg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Comenzamos con la subida de los datos transformados

In [3]:
with open('./Datos/Transformados/df_transformado.csv', 'r') as fichero:
    csv_dict = DictReader(fichero)
    indice = 0
    for row in csv_dict:
        row['booking_time'] = pd.to_datetime(row['booking_time'])
        row['checkin_time'] = pd.to_datetime(row['checkin_time'])
        row['checkout_time'] = pd.to_datetime(row['checkout_time'])
        row['cancelled_imp'] = pd.to_datetime(row['cancelled_imp'])
        row['adult_count'] = int(row['adult_count'])
        row['child_count'] = int(row['child_count'])
        row['total_personas'] = int(row['total_personas'])
        row['age'] = int(float(row['age']))
        row['recurrence'] = int(row['recurrence'])
        row['product_count'] = int(row['product_count'])
        row['reservation_net_value'] = float(row['reservation_net_value'])
        row['reservation_nights_net_value'] = float(row['reservation_nights_net_value'])
        row['extras_net_value'] = float(row['extras_net_value'])
        row['dif_can_in'] = int(row['dif_can_in'])
        row['dif_can_res'] = int(row['dif_can_res'])
        row['dif_ent_res'] = int(row['dif_ent_res'])
        row['duracion_reserva'] = int(row['duracion_reserva'])
        if row['cancellation'] == 'True':
            row['cancellation'] = 'Verdadero'
            row['cancellation'] = bool(row['cancellation'])
            row['cancelled_at'] = pd.to_datetime(row['cancelled_at'], errors='coerce', format = '%Y-%m-%dT%H:%M:%S')
            resp = client.index(index = 'reto06_morado_datos_limpios_01', id = indice, document=row)
            indice = indice + 1
        elif row['cancellation'] == 'False':
            row['cancellation'] = bool(row['cancellation'])
            row['cancelled_at'] = dt.now()
            resp = client.index(index = 'reto06_morado_datos_limpios_01', id = indice, document=row)
            indice = indice + 1

In [4]:
resp = client.search(index="reto06_morado_datos_limpios_01", query={"match_all": {}})
indice1 = resp['hits']['total']['value']
print(f'Se han indexado {indice1}')

Se han indexado 10000


In [5]:
with open('./Datos/Transformados/df_transformado.csv', 'r') as fichero:
    csv_dict = DictReader(fichero)
    indice = indice1
    acum = 0
    for row in csv_dict:
        acum = acum + 1
        if acum>indice1:
            row['booking_time'] = pd.to_datetime(row['booking_time'])
            row['checkin_time'] = pd.to_datetime(row['checkin_time'])
            row['checkout_time'] = pd.to_datetime(row['checkout_time'])
            row['cancelled_imp'] = pd.to_datetime(row['cancelled_imp'])
            row['adult_count'] = int(row['adult_count'])
            row['child_count'] = int(row['child_count'])
            row['total_personas'] = int(row['total_personas'])
            row['age'] = int(float(row['age']))
            row['recurrence'] = int(row['recurrence'])
            row['product_count'] = int(row['product_count'])
            row['reservation_net_value'] = float(row['reservation_net_value'])
            row['reservation_nights_net_value'] = float(row['reservation_nights_net_value'])
            row['extras_net_value'] = float(row['extras_net_value'])
            row['dif_can_in'] = int(row['dif_can_in'])
            row['dif_can_res'] = int(row['dif_can_res'])
            row['dif_ent_res'] = int(row['dif_ent_res'])
            row['duracion_reserva'] = int(row['duracion_reserva'])
            if row['cancellation'] == 'True':
                row['cancellation'] = 'Verdadero'
                row['cancellation'] = bool(row['cancellation'])
                row['cancelled_at'] = pd.to_datetime(row['cancelled_at'], errors='coerce', format = '%Y-%m-%dT%H:%M:%S')
                resp = client.index(index = 'reto06_morado_datos_limpios_2', id = indice, document=row)
                indice = indice + 1
            elif row['cancellation'] == 'False':
                row['cancellation'] = bool(row['cancellation'])
                row['cancelled_at'] = dt.now()
                resp = client.index(index = 'reto06_morado_datos_limpios_2', id = indice, document=row)
                indice = indice + 1
        else:
            continue

In [6]:
resp = client.search(index="reto06_morado_datos_limpios_2", query={"match_all": {}})
indice2 = resp['hits']['total']['value']
print(f'Se han indexado {indice2}')

Se han indexado 582


In [7]:
print(f'Se han indexado en total en los dos indices {indice1 + indice2}')

Se han indexado en total en los dos indices 10582
